## Building MultiModal Search with Vector Databases 

In [ ]:
%env PALM_API_KEY=ya29.here...

## Connect to Weaviate

In [ ]:
import weaviate, os

# Connect to a Weaviate Cloud instance
client = weaviate.connect_to_wcs(
    cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_ADMIN")),
    
    headers={
        "X-PALM-Api-Key": os.getenv("PALM_API_KEY"),
    }
)

client.is_ready()

## Helper functions

In [ ]:
# Helper functions to display results
import json
from IPython.display import Image, Video

def json_print(data):
    print(json.dumps(data, indent=2))

def display_media(item):
    path = item["path"]

    if(item["mediaType"] == "image"):
        display(Image(path, width=300))

    elif(item["mediaType"] == "video"):
        display(Video(path, width=300))

In [ ]:
import base64, requests

# Helper function – get base64 representation from an online image
def url_to_base64(url):
    image_response = requests.get(url)
    content = image_response.content
    return base64.b64encode(content).decode('utf-8')

# Helper function - get base64 representation from a local file
def file_to_base64(path):
    with open(path, 'rb') as file:
        return base64.b64encode(file.read()).decode('utf-8')

# Update the url and path to test
#test_image_base64 = url_to_base64("https://path-to-some-online-image.jpg")
#test_file_base64 = file_to_base64("./test/meerkat.jpeg")

## Text to Media Search

In [ ]:
animals = client.collections.get("Animals")

response = animals.query.near_text(
    query="dog with stick",
    return_properties=['name','path','mediaType'],
    limit=3
)

In [ ]:
for obj in response.objects:
    json_print(obj.properties)
    display_media(obj.properties)

## Image to Media Search

In [ ]:
Image("./test/test-cat.jpg", width=300)

In [ ]:
response = animals.query.near_image(
    near_image=file_to_base64("./test/test-cat.jpg"),
    return_properties=['name','path','mediaType'],
    limit=3
)

for obj in response.objects:
    json_print(obj.properties)
    display_media(obj.properties)

## Image search - from web URL

In [ ]:
Image("https://raw.githubusercontent.com/weaviate-tutorials/multimodal-workshop/main/2-multimodal/test/test-meerkat.jpg", width=300)

In [ ]:
response = animals.query.near_image(
    near_image=url_to_base64("https://raw.githubusercontent.com/weaviate-tutorials/multimodal-workshop/main/2-multimodal/test/test-meerkat.jpg"),
    return_properties=['name','path','mediaType'],
    limit=3
)

for obj in response.objects:
    json_print(obj.properties)
    display_media(obj.properties)

## Video to Media Search
> Note. the input video must be at least 4 seconds long

In [ ]:
Video("./test/test-meerkat.mp4")

In [ ]:
from weaviate.classes.query import NearMediaType

response = animals.query.near_media(
    media=file_to_base64("./test/test-meerkat.mp4"),
    media_type=NearMediaType.VIDEO,
    return_properties=['name','path','mediaType'],
    limit=3
)

for obj in response.objects:
    # json_print(obj.properties)
    display_media(obj.properties)